In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model,Sequential
from keras.datasets import mnist
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import adam

In [11]:
def load_data():
    (x_train,y_train),(x_test,y_test) = mnist.load_data()
    x_train = (x_train.astype(np.float32)-127.5)/127.5
    x_train = x_train.reshape(60000,784)
    return(x_train,y_train,x_test,y_test)

(X_train,Y_train,X_test,Y_test) = load_data()
print(X_train.shape)
    

(60000, 784)


In [19]:
def adam_optimizer():
    return keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

In [22]:
def create_generator():
    generator = Sequential()
    generator.add(Dense(units = 256,input_dim=100))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=512))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=784,activation='tanh'))
    
    generator.compile(loss='binary_crossentropy',optimizer=adam_optimizer())
    return generator

g = create_generator()
g.summary()
                  

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
dense_32 (Dense)             (None, 784)              

In [23]:
def create_discriminator():
    discriminator = Sequential()
    discriminator.add(Dense(units = 1024,input_dim=784))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(units = 512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(units = 256))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.compile(loss='binary_crossentropy',optimizer=adam_optimizer())
    return discriminator

d = create_discriminator()
d.summary()
    
    
    

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_34 (Dense)             (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 256)              

In [26]:
def create_gan(discriminator,generator):
    discriminator.trainable = False
    gan_input = Input((100,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(inputs = gan_input , outputs=gan_output)
    gan.compile(loss='binary_crossentropy',optimizer='adam')
    return gan

gan = create_gan(d,g)
gan.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_8 (Sequential)    (None, 784)               1486352   
_________________________________________________________________
sequential_9 (Sequential)    (None, 256)               1459968   
Total params: 2,946,320
Trainable params: 1,486,352
Non-trainable params: 1,459,968
_________________________________________________________________
